# Stock Trading Strategy Based on Leading Indicators

I don't know much about stock trading strategies. The reason I created this notebook is to play around with Python data science tools and the stock market has lots of data. Also I wanted something "actionable" if the idea showed promise.

One of the reasons I have not looked into stock trading strategies is I believe in the ["Efficient Market Hypothesis (EMH)"](https://www.investopedia.com/terms/e/efficientmarkethypothesis.asp). This basically says that you can't make money off well known trading strategies because everyone uses them, which causes the price to change, and erases the potential gains. By the time Jim Cramer is talking about it, you are too late. Or worse you are about to be the victim of a pump-and-dump operation.

Then again, as a programmer, I have always been fascinated by the ["combinatorial explosion"](https://en.wikipedia.org/wiki/Combinatorial_explosion); as the number of things there is to combine grows, the number of possible combinations grows exponentially. In the stock market, its plausible that some combination of financial measures could predict short-term changes in a given stock's price. Since there are so many possible combinations of financial measures, it seems possible to find one that no one else knows about; thus avoiding the EMH.

Of course, when searching such a large space of possible measures, it is likely that many of these will be actually be the results of over-fitting noise. In those cases, future stock prices will not be correlated with the predictors. On average, the return of the strategy will be 0, minus the trading costs. For now, we will assume the trading costs are 0, so the costs of over-fitting will just be the opportunity cost of not investing in a better performing strategy.

## The Strategy
The goal is to predict short-term price spikes in a target stock based on patterns in other finacial measures. To do this, we will:

1. pick a target stock
2. find all the price spikes in the recent historical data of that stock
3. look for patterns in the short-term historical data of many other financial measures, right before the spikes in the target
4. combine them to make a predictor for that stock

## The Financial Measures
Their are many possible financial measures; gold, oil, sentiment analysis of Reddit, ... To make things easy for now, we will just use historical stock data that can be obtained from Yahoo, for free. More specifically, the NASDAQ.

## Spikes
We want the spikes to be something we can make money off of. It seems like it will be easier to predict short-term changes. We will try this pattern: look for a sequence of 3 prices that are monotonically increasing and have a gain that is greater than 3% (e.g. p[t+2] > p[t+1] > p[t] and (p[t+2] - p[t])/p[t] > 0.03).

## Estimating the Predictor Function
We will assume that in some short time window, immediately before each spike<sub>i</sub>, the historical data for a single stock will be:

$h_{i}(t) = f(t)(a_{i}p(t) + r(t))$

where f(t) is the signal in the predictor stock that would have happened if the economic event that caused the spike in the target did not happen, p(t) is the predictor function for that stock, a<sub>i</sub> is a scalar and r(t) is random noise. 

Extracting p(t) is only possible with some simplifying assumptions. The perennial favorite is make f(t) piecewise linear by doing a Taylor series expansion. But even linear is kind of messy. 

In the year after (covid) April 2020, there was an almost unprecedented bull market. During that time, the S&P 500 gained about 12%, giving a weekly gain of about 0.2%. Looks like we can ignore the linear term. Lets just say that f(t) is a constant before each spike.

$h_{i}(t) = b_{i}(a_{i}p(t) + r(t))$

Averaging $h_{i}(t)$ would give the shape of p(t), but it would excessively weight events that happened when the predictor stock price was high. So we will divide each $h_{i}(t)$ by the average value during the time period before the spike.

## The Stock Data

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

import leading_pattern.model as model
import leading_pattern.by_yield as by_yield
import leading_pattern.roc_auc as roc_auc
import leading_pattern.plotting as plotting
import my_utils.loaders as loaders

plt.rcParams['figure.figsize'] = [8, 5]
np.seterr(all='raise')  # make code stop on warnings

# Two years of historical data. The first year is for training, the second year is for testing.
CONFIG = {
    'start_date': '20170103',
    'split_date': '20171231',
    'end_date': '20181231',
    'spike_length': 2,
    'spike_threshold_percent': 3.0,
    'predictor_func_length': 7,
    'use_diffs': True,
    'price_field': 'Open',
    'predictor_field': 'Open'
}

# We can get a CSV file of all the stocks in the NASDAQ from here: https://www.nasdaq.com/market-activity/stocks/screener
# Lets read all the stocks into a pandas dataframe
all_stocks = model.read_nasdaq_stock_list()

print(all_stocks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7611 entries, 0 to 7610
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7611 non-null   object 
 1   Name        7611 non-null   object 
 2   Last Sale   7611 non-null   object 
 3   Net Change  7611 non-null   float64
 4   % Change    7598 non-null   object 
 5   Market Cap  6987 non-null   float64
 6   Country     6951 non-null   object 
 7   IPO Year    4341 non-null   float64
 8   Volume      7611 non-null   int64  
 9   Sector      5502 non-null   object 
 10  Industry    5501 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 654.2+ KB
None


In any list like this, there is bound to be some oddballs. Lets look at some summary statistics.

But first lets fix "Last Sale", which is the price of the last sale. The values have a leading $, so pandas treats it as a string. Lets convert it to a float. Lets also add a column that is the "Last Sale" x "Volume".

In [ ]:
all_stocks['Last Sale'] = all_stocks['Last Sale'].apply(lambda x: float(x.replace('$','')))
all_stocks['dollar_vol'] = all_stocks['Last Sale']* all_stocks['Volume']
all_stocks.describe()

Last-Sale is the price of the last trade. Look at that max! Almost a half million. IPO Year is a little odd because Yahoo reader has convert the int into a float. We will exclude stocks that had an IPO Year after our analysis start year minus one year.

Volume seems like might be a reasonable field for removing outliers. To speed up the exploratory analysis, lets limit the number of predictor stocks to a random sample of 10, whose volume is above the 0.1 quantile.

In [ ]:
analysis_start_year = float(CONFIG['start_date'][0:4])
symbols = model.select_random_stock_symbols(all_stocks, 10, analysis_start_year, quantile=0.1, filter_field='Volume')

# Use these from a previous random sample
symbols = ['NPTN', 'MTL', 'AGR', 'PHD', 'ESS', 'CBOE', 'AGRO', 'FPF', 'CHMA', 'SEAS']

data_sets = loaders.get_data_sets(symbols, CONFIG)

# Lets select the target stock and look and it's columns
target_stock = symbols[0]  # we will try to predict the first one
print(data_sets['train'][target_stock].info())

The first 6 columns are primary data. It's not clear which to use for this project. One thought might be to use them all; maybe in a neural net? But there is a problem...

## Water Everywhere, Nothing to Drink
Although we have lots of data, we do not have enough of the right kind of data to use every tool in the machine learning toolbox. The big problem is that the stock market and the economy are constantly changing. It's not clear if the patterns in the data from 5 years ago are still relevant data now.

For now, we are using historical data from the two years before covid.

Also, we chose to limit our analysis to the daily opening price. While we could get more data by trying to get hourly data, that was not readily available. Moreover, when the time interval between data points is small enough we might run into other problems. Stock prices are set by trades and trades are discrete causing additional noise. And for a somewhat smooth continuous function, nearby values are highly correlated.

## Back to the Historical Stock Data
Lets look at the spikes in the target stock in the training data.

In [ ]:
df = data_sets['train'][target_stock]
model.find_spikes(df, CONFIG)  # Find spikes in target stock and a column 'spikes' to dataframe.
plotting.plot_spikes(df, CONFIG['price_field'])

With 28 spikes, it looks like this target stock has enough volatility to train on. Lets load snippets of historical data in each predictor stock before each spike in the target stock. Note, we are also using the target stock as a predictor of its self.

In [ ]:
CONFIG['use_diffs'] = False  # the reason for this will be clear in a moment
predictor_func_by_stock = model.get_predictor_func_by_stock(data_sets['train'], target_stock, symbols, CONFIG)
plotting.plot_predictor_functions(symbols, target_stock, predictor_func_by_stock)

It is surpising that these are so similar. We checked; it's not a bug. Although each predictor function is created from a different stock, they all are created from the same time points. What we are seeing is that all these stocks follow a similar trend. Below are the predictor functions derived from the same number of spikes at random times:

In [ ]:
# Make spikes at random times
df = data_sets['train'][target_stock]
spikes = df.spikes.copy()  # save so we can restore for further analysis
spike_count = df['spikes'][df['spikes']].shape[0]
random_spikes = np.full_like(df.spikes, False)
indices = np.random.randint(0, df['spikes'].shape[0] - 1, spike_count)
random_spikes[indices] = True
df.spikes = random_spikes

predictor_func_by_stock = model.get_predictor_func_by_stock(data_sets['train'], target_stock, symbols, CONFIG)
plotting.plot_predictor_functions(symbols, target_stock, predictor_func_by_stock)
df.spikes = spikes  # restore

All these predictor functions are similar as well, implying that the shape of the curves is mostly determined by the times of the spikes, not a pattern unique to each predictor stock.
 
Our hypothesis is that over a time span of a few days, market information ripples through the predictor stocks before the spike in the target stock. To look for these ripples we should de-trend the data by taking the difference of successive time points.

In [ ]:
CONFIG['use_diffs'] = True
predictor_func_by_stock = model.get_predictor_func_by_stock(data_sets['train'], target_stock, symbols, CONFIG)
plotting.plot_predictor_functions(symbols, target_stock, predictor_func_by_stock)

Now the predictor functions look different for each predictor, which is what we expected.

## Calculating Performance of Each Predictor
The objective of this strategy is not neccessarily to make correct predictions. Rather it is to make money. Lets try running the predictors and look at the price changes after each positive prediction (true positive and false positive) as a function of threshold of $r^2$

One way to do this is to just add up all the price changes to get the total yield. The problem with that approach is it combines the change from each trade with the number of trades. To remove the effect of trading frequency, we will calculate the expected-mean-percent-change.

To calculate expected-mean-percent-change, we start by calculating all the percent changes in the test data. If the predictor did not predict the price then it would be like randomly sampling from these changes (null hypothesis). Next we set the $r^2$ threshold to predict when to buy which creates another sample of the percent changes. We use the Kolmogorov-Smirnov to calculate the probability (p) that our sample is the same as the random sample. Then we calculate expected mean percent change (mpc):

$expected\ mpc = p * mpc_{null}+(1-p)*mpc_{predictor}$

$\sigma_{empc}=\sqrt{p^2*\sigma_{null}^2+(1-p)^2*\sigma_{predictor}^2}$

In [ ]:
df_by_predictor = by_yield.calc_performance_by_threshold_for_predictors(target_stock, symbols, CONFIG, data_set_name='test')
plotting.plot_calc_performance_scores_by_threshold_for_predictors(target_stock, df_by_predictor, CONFIG['predictor_field'])

We plot low $r^2$ thresholds to show effectively random predictions. If a predictor worked, we would expect the optimal $r^2$ threshold to be higher.

None of these predictors look great. This is not too surprising; most stocks are not strong predictors of other stocks, much less 10 randomly selected stocks. Further, the target stock itself might not be predictable using this strategy.

Just for fun, we can run the code using a different data frame column without having to change the code. Lets try "Volume".

In [ ]:
CONFIG['predictor_field'] = 'Volume'
df_by_predictor = by_yield.calc_performance_by_threshold_for_predictors(target_stock, symbols, CONFIG, data_set_name='test')
plotting.plot_calc_performance_scores_by_threshold_for_predictors(target_stock, df_by_predictor, CONFIG['predictor_field'])

We only calculate performance for $r^2$ thresholds where there are 5 or more predictions. That is why the threshold range is is truncated for some predictors.

The only predictor that might be worth looking into is PHD.

## Back to Reality
In many machine learning projects, there is plenty of data and the system being modelled is stationary. This means that the data can be split into "train", "dev" and "test". We could use "train" to estimate the predictor functions. "dev" to pick the threshold and the predictor stocks. Then use "test" to see how well this works.

Unfortunately our data is limited and not necessarily stationary. We will have to cut corners. To start, lets look at mean-percent-change on the training data. This is cheating since we are running the predictor functions on the data that generated them. But it is useful cheating because it gives us an upper bound on how well the strategy could work.

In [ ]:
CONFIG['predictor_field'] = 'Open'
df_by_predictor = by_yield.calc_performance_by_threshold_for_predictors(target_stock, symbols, CONFIG, data_set_name='train')
plotting.plot_calc_performance_scores_by_threshold_for_predictors(target_stock, df_by_predictor, CONFIG['predictor_field'])

If we were to select predictor stocks based on training data, we might conclude that SEAS is a good predictor. Yet as we saw above, "test" shows that it is not a predictor at all.

We ran all predictors and sorted the results based on the yield in the "training" data. The results with the 10 highest expected-mean-percent-change are shown below.

In [ ]:
df = by_yield.compare_train_test(target_stock, CONFIG)
print(df)

Evidently the training data is not a very good predictor of the test data. Maybe AXS is a good predictor, but if we are only looking at the training data performance, how would we select that and not LCI? There are avenues to explore. Maybe add a "dev" set. Maybe look at how similar the data snippets are that are used to create the predictor func.

But there is another thing to consider. Our pragmatic use of mean-percent-change combines the strategy performance with the target stock prices. [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) is a way to look at the strategy in isolation. 

In [ ]:
data_set_name = 'test'
predictor_func_by_stock = model.get_predictor_func_by_stock(data_sets['train'], target_stock, symbols, CONFIG)
auc_df_by_predictor = {}
for predictor, p_func in predictor_func_by_stock.items():
    model.calc_r_squared_for_list(data_sets[data_set_name], predictor_func_by_stock, CONFIG)
    auc_df_by_predictor[predictor] = \
        roc_auc.calc_fpr_tpr(data_sets[data_set_name][target_stock], data_sets[data_set_name][predictor], CONFIG)
plotting.plot_roc_panel(auc_df_by_predictor, target_stock, CONFIG)

None of these look very promising.

We ran this analysis for all predictors and sorted by the area-under-the-curve (AUC) for the training data. Once again, the training performance is not a good predictor of the test data.

In [ ]:
df = roc_auc.summarize_auc(target_stock, CONFIG, max_rows=10)
print(df)

It looks like NOMD is the best of the not-very-good. Lets say we somehow selected that as a predictor. How might it perform?

In [ ]:
predictor = 'NOMD'
data_sets2 = loaders.get_data_sets([target_stock, predictor], CONFIG)
predictor_func_by_stock2 = model.get_predictor_func_by_stock(data_sets2['train'], target_stock, [predictor], CONFIG)
model.calc_r_squared_for_list(data_sets2['test'], predictor_func_by_stock2, CONFIG)
auc_df = roc_auc.calc_fpr_tpr(data_sets2['test'][target_stock], data_sets2['test'][predictor], CONFIG)
print(auc_df)

It's not clear what threshold is optimal. For sake of argument, lets say 0.55 is best and that tpr=10% with fpr=5%. For NPTN, there a about 30 real spikes in a year. That means we would get 3 true positives and 1.5 false positives. Note, if we select the next higher threshold, the fpr is about twice the tpr and the tpr drops dramatically. Selecting a threshold is clearly important.

To double our money, we need to have about 25 true positives in a year. So we would need about 8 stocks like NPTN. That might be possible if we could find a robust method for selecting predictors and thresholds.

## Next Steps
If we wanted to continue with this strategy, then next step would be to find a way to pick the predictors and thresholds using the shortest time span possible. This could involve including a split just for that. Maybe 10 months for training, 3 months for dev and 3 months for test. Maybe interleave the splits.

Another option would be to look at how the predictor function converges. The predictor function is made by averaging many snippets. If many of the snippets looked similar, then the average might be meaningful. Whereas if each snippet looked completely different, we would still get an average, but it would just be an average of noise.

Or maybe a completely different strategy...